# DSCI 100 Group Project Proposal - Group 17

In [1]:
library(tidyverse)
library(tidymodels)
library(readr)
library(dplyr)

set.seed(999)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

## Introduction

* Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
* Clearly state the question you will try to answer with your project
* Identify and describe the dataset that will be used to answer the question

Data source: https://archive.ics.uci.edu/ml/datasets/Audit+Data

Publication: https://www.researchgate.net/publication/323655455_Fraudulent_Firm_Classification_A_Case_Study_of_an_External_Audit

## Preliminary Exploratory Data Analysis

* Demonstrate that the dataset can be read from the web into R 
* Clean and wrangle your data into a tidy format
* Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
* Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.


**Dataset Info**
| | Inherent Risk Factors | | Control Risk Factors |
| --- | --- | ---| --- |
| ParaA | Discrepancy in planned-expenditure of inspection and summary report A in Rs | Sector score | Historical risk score value of the target-unit |
| ParaB | Discrepancy in unplanned-expenditure of inspection and summary report B in Rs | Loss | Amount of loss suffered by the firm last year. |
| Total | Total amount of discrepancy found in other reports Rs | History | Average historical loss suffered by firm in the last 10 years. |
| Number | Historical discrepancy score. | District score | Historical risk score of a district in the last 10 years. |
| Money Value | Amount of money involved in misstatements in the past audits. | | |
| Sector ID | Unique ID of the target sector. | Location ID | Unique ID of the city/province. |
| ARS | Total risk score using analytical procedure. | Audit ID | Unique Id assigned to an audit case. |
| Risk class | Risk Class assigned to an audit-case, 1=fraud or 0=no fraud (Target Feature) 

In [51]:
# import the data set, make column names consistent
url <- "https://raw.githubusercontent.com/hakkd/dsci-100-project-group17/main/trial.csv"

colnames = c("sector_score", "location_id", "para_a", "score_a", "para_b", "score_b", 
             "total", "numbers", "marks", "money_value", "money_marks", "district", "loss", 
             "loss_score", "history", "history_score", "score", "risk")

audit_data <- read_csv(url, col_names = colnames)

head(audit_data)

Rows: 777 Columns: 18
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (18): sector_score, location_id, para_a, score_a, para_b, score_b, total...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


sector_score,location_id,para_a,score_a,para_b,score_b,total,numbers,marks,money_value,money_marks,district,loss,loss_score,history,history_score,score,risk
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Sector_score,LOCATION_ID,PARA_A,SCORE_A,PARA_B,SCORE_B,TOTAL,numbers,Marks,Money_Value,MONEY_Marks,District,Loss,LOSS_SCORE,History,History_score,Score,Risk
3.89,23,4.18,6,2.5,2,6.68,5,2,3.38,2,2,0,2,0,2,2.4,1
3.89,6,0,2,4.83,2,4.83,5,2,0.94,2,2,0,2,0,2,2,0
3.89,6,0.51,2,0.23,2,0.74,5,2,0,2,2,0,2,0,2,2,0
3.89,6,0,2,10.8,6,10.8,6,6,11.75,6,2,0,2,0,2,4.4,1
3.89,6,0,2,0.08,2,0.08,5,2,0,2,2,0,2,0,2,2,0


In [35]:
# change Risk to factor and recode as F (fraud) or N (no fraud)
audit_data <- audit_data |>
        mutate(risk = as.factor(risk))

audit_data$risk <- ifelse(audit_data$risk == "1", "F", "N")

head(audit_data)

Sector_score,LOCATION_ID,PARA_A,SCORE_A,PARA_B,SCORE_B,TOTAL,numbers,Marks,Money_Value,MONEY_Marks,District,Loss,LOSS_SCORE,History,History_score,Score,Risk
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
3.89,23,4.18,6,2.50,2,6.68,5,2,3.38,2,2,0,2,0,2,2.4,F
3.89,6,0.00,2,4.83,2,4.83,5,2,0.94,2,2,0,2,0,2,2.0,N
3.89,6,0.51,2,0.23,2,0.74,5,2,0.00,2,2,0,2,0,2,2.0,N
3.89,6,0.00,2,10.80,6,10.80,6,6,11.75,6,2,0,2,0,2,4.4,F
3.89,6,0.00,2,0.08,2,0.08,5,2,0.00,2,2,0,2,0,2,2.0,N
3.89,6,0.00,2,0.83,2,0.83,5,2,2.95,2,2,0,2,0,2,2.0,N


In [45]:
colnames(audit_data)

[1] "Sector_score"  "LOCATION_ID"   "PARA_A"        "SCORE_A"      
 [5] "PARA_B"        "SCORE_B"       "TOTAL"         "numbers"      
 [9] "Marks"         "Money_Value"   "MONEY_Marks"   "District"     
[13] "Loss"          "LOSS_SCORE"    "History"       "History_score"
[17] "Score"         "Risk"

In [4]:
# creating training and testing datasets
audit_split <- initial_split(audit_data, prop = 0.75, strata = Risk)

audit_train <- training(audit_split)
audit_test <- testing(audit_split)

In [7]:
# check proportion of fraudulent and not fraudulent cases
audit_proportions <- audit_train |>
                      group_by(Risk) |>
                      summarize(n = n()) |>
                      mutate(percent = 100*n/nrow(audit_train))

audit_proportions

Risk,n,percent
<dbl>,<int>,<dbl>
0,217,37.3494
1,364,62.6506


Based on the table above, this dataset has more fraudulent observations than not-fraudulent ones (~2:1)

In [12]:
# show summary statistics for all variables
summary(audit_train)

  Sector_score   LOCATION_ID            PARA_A          SCORE_A     
 Min.   : 1.85   Length:581         Min.   : 0.000   Min.   :2.000  
 1st Qu.: 2.37   Class :character   1st Qu.: 0.200   1st Qu.:2.000  
 Median : 3.89   Mode  :character   Median : 0.850   Median :2.000  
 Mean   :20.49                      Mean   : 2.522   Mean   :3.501  
 3rd Qu.:55.57                      3rd Qu.: 2.430   3rd Qu.:6.000  
 Max.   :59.85                      Max.   :85.000   Max.   :6.000  
     PARA_B           SCORE_B          TOTAL            numbers     
 Min.   :   0.00   Min.   :2.000   Min.   :   0.00   Min.   :5.000  
 1st Qu.:   0.00   1st Qu.:2.000   1st Qu.:   0.48   1st Qu.:5.000  
 Median :   0.35   Median :2.000   Median :   1.37   Median :5.000  
 Mean   :  11.58   Mean   :3.105   Mean   :  14.06   Mean   :5.074  
 3rd Qu.:   4.20   3rd Qu.:4.000   3rd Qu.:   7.69   3rd Qu.:5.000  
 Max.   :1264.63   Max.   :6.000   Max.   :1268.91   Max.   :9.000  
     Marks        Money_Value     

## Methods

* Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
* Describe at least one way that you will visualize the results

## Expected Outcomes and Significance

* What do you expect to find?
* What impact could such findings have?
* What future questions could this lead to?
